#### Librerías necesarias

In [ ]:
import os
import ast
import base64
import logging
import requests
import matplotlib.pyplot as plt

from dotenv import load_dotenv
from notion_client import Client

#### Dotenv

In [ ]:
# Cargar variables .env
load_dotenv()

#### Logger

In [ ]:
# Logger
logging.basicConfig(format='[%(asctime)s]:[%(levelname)s]:[%(filename)s %(lineno)d]:[%(funcName)s]:%(message)s')
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

#### Autenticación con Graph API

In [ ]:
# Configuración de Graph API
TENANT_ID = os.getenv('TENANT_ID')
CLIENT_ID = os.getenv('CLIENT_ID')
PAYLOAD = os.getenv('PAYLOAD')
COOKIE = os.getenv('COOKIE')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
REFRESH_TOKEN = os.getenv('REFRESH_TOKEN')
GRAPH_API_ENDPOINT = 'https://graph.microsoft.com/v1.0/me/sendMail'

url = f'https://login.microsoftonline.com/{TENANT_ID}/oauth2/v2.0/token'
payload = PAYLOAD
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
  'Cookie': f'{COOKIE}; stsservicecookie=estsfd; x-ms-gateway-slice=estsfd'
}

try:
    response = requests.post(url, headers=headers, data=payload)
    response.raise_for_status()

    result = ast.literal_eval(str(response.text))

except requests.exceptions.RequestException as e:
    logger.exception("Ocurríó un error mientras se solicitaba el token.")

#### Conexión al cliente de Notion y parámetros

In [ ]:
NOTION_TOKEN = os.getenv('NOTION_TOKEN')
PRODUCTS_DATABASE_ID = os.getenv('PRODUCTS_DATABASE_ID')
TASKS_DATABASE_ID = os.getenv('TASKS_DATABASE_ID')
USERS_DATABASE_ID = os.getenv('USERS_DATABASE_ID')

# Iniciar cliente
notion = Client(auth = NOTION_TOKEN)

## Base de datos

#### Gestores

In [ ]:
# Query a base de datos Gestores
users_db = notion.databases.query(
    **{
        "database_id": USERS_DATABASE_ID,
    }
).get("results")

# Diccionario que contendrá los ID de los gestores junto a sus nombres y apodos
users = {}

for query in users_db:
    user_id = query.get("id")
    user_name = query.get("properties").get("Nombre").get("title")[0].get("plain_text")
    user_nickname = query.get("properties").get("Nombre de usuario").get("rich_text")[0].get("plain_text")
    user_email = query.get("properties").get("Correo electrónico").get("email")
    users[user_id] = {
        "name": user_name,
        "nickname": user_nickname,
        "email": user_email
    }

# Query a base de datos Tareas
tasks_db = notion.databases.query(
    **{
        "database_id": TASKS_DATABASE_ID,
    }
).get("results")

#### Tareas

In [ ]:
# Query a base de datos Tareas
tasks_db = notion.databases.query(
    **{
        "database_id": TASKS_DATABASE_ID,
    }
).get("results")

# Lista que contendrá los id de los gestores junto a sus nombres
tasks = {}

for query in tasks_db:
    task_id = query.get("id")
    task_name = query.get("properties").get("Nombre de la tarea").get("title")[0].get("plain_text")
    task_status = query.get("properties").get("Estado").get("status").get("name")
    try:
        task_due_date = query.get("properties").get("Fecha fin. planificada").get("date").get("start")
    except AttributeError:
        task_due_date = "No definido"
    tasks[task_id] = {
        "name": task_name,
        "status": task_status,
        "due_date": task_due_date
    }

#### Productos

In [ ]:
# Query a base de datos Productos
products_db = notion.databases.query(
    **{
        "database_id": PRODUCTS_DATABASE_ID,
    }
).get("results")

data = []

for query in products_db:
    products = {}
    product_id = query.get("id")
    product_name = query.get("properties").get("Nombre del producto").get("title")[0].get("plain_text")
    product_user_id = query.get("properties").get("Gestor(es)").get("relation")[0].get("id")
    product_user = users.get(product_user_id, {"name": "No definido", "nickname": "No definido", "email": "No definido"})
    try:
        product_status = query.get("properties").get("Estado").get("formula").get("string")
    except AttributeError:
        product_status = "Estado desconocido"
    product_tasks_id = [item.get("id") for item in query.get("properties").get("Tareas").get("relation")]
    product_tasks = []
    for task_id in product_tasks_id:
        task_info = tasks.get(task_id)
        if task_info:
            product_tasks.append(task_info)
    product_progress = query.get("properties").get("Progreso").get("rollup").get("number") or 0
    if product_progress:
        product_progress *= 100
    try:
        product_priority = query.get("properties").get("Prioridad").get("select").get("name")
    except AttributeError:
        product_priority = "Prioridad no definida"
    try:
        product_end_date = query.get("properties").get("Fecha planificada").get("date").get("end")
    except AttributeError:
        product_end_date = "Fecha de finalización no definida"

    products["Nombre del producto"] = product_name
    products["Prioridad"] = product_priority
    products["Estado"] = product_status
    products["Usuario"] = product_user
    products["Progreso"] = product_progress
    products["Tareas"] = product_tasks
    products["Fecha fin planificada"] = product_end_date
    data.append(products)

### Reporte de rendimiento

In [ ]:
# Agrupar los datos por usuario
user_reports = {}
for report in data:
    user_name = report["Usuario"]["name"]
    if user_name not in user_reports:
        user_reports[user_name] = []
    user_reports[user_name].append(report)

#### Función para generar el gráfico

In [ ]:
# Función para generar el gráfico
def generate_graph(reports, user_name, nickname, output_dir):
    product_name = [report['Nombre del producto'] for report in reports]
    product_progress = [report.get('Progreso', 0) for report in reports]

    fig, ax = plt.subplots()
    ax.barh(product_name, product_progress, color='skyblue')
    ax.set_xlabel('Tareas realizadas (%)')
    ax.set_title(f'Progreso de productos de {user_name}')

    plt.xticks(rotation=45, ha='right')
    plt.yticks(fontsize=8)
    plt.tight_layout(rect=[0, 0, 1, 1])

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    graph_path = os.path.join(output_dir, f'{nickname.replace(" ", "_").lower()}.png')
    plt.savefig(graph_path, bbox_inches='tight', dpi=300)
    plt.close()
    return graph_path

#### Función para generar el HTML

In [ ]:
# Función para generar el HTML
def generate_html(user_name, nickname, reports, graph_path, template_path, output_dir):
    with open(template_path, 'r', encoding='utf-8') as file:
        template = file.read()

    none_count = 0
    no_started_count = 0
    in_progress_count = 0
    completed_count = 0
    
    for count in reports:
        if count.get("Estado") is None:
            none_count += 1
        elif count.get("Estado") == 'Sin empezar':
            no_started_count += 1
        elif count.get("Estado") == 'En curso':
            in_progress_count += 1
        elif count.get("Estado") == 'Listo':
            completed_count += 1

    num_products = len(reports)
    body = f'Hasta la fecha has registrado {num_products} productos. De estos, '
    
    status_messages = []
    
    if none_count > 0:
        status_messages.append(f'{none_count} no tienen ninguna tarea asignada')
    
    if no_started_count > 0:
        status_messages.append(f'{no_started_count} no han empezado')
    
    if in_progress_count > 0:
        status_messages.append(f'{in_progress_count} están en curso')
    
    if completed_count > 0:
        status_messages.append(f'{completed_count} están completados')
    
    if status_messages:
        body += ', '.join(status_messages[:-1])
        if len(status_messages) > 1:
            body += ' y '
        body += status_messages[-1]
    else:
        body += 'no hay información disponible.'
        
    body += '.<br><br>'
    body += 'Si te aparece Progreso en 0% y Tareas restantes en 0 significa que aún no has registrado tareas a dicho producto.<br><br>'
    body += 'No olvides <b>actualizar periódicamente</b> la información en Notion. A continuación, se muestra la lista de productos que están <b>en curso</b>:<br><br>'

    for report in reports:
        if report["Estado"] != 'Listo':
            body += f'<strong>{report["Nombre del producto"]}</strong><br>'
            body += f'Prioridad: {report["Prioridad"]}<br>'
            body += f'Progreso: {report.get("Progreso", 0)}%<br>'
            body += f'Fecha fin planificada: {report.get("Fecha fin planificada", "No definida")}<br>'
            body += f'Tareas restantes: {len([t for t in report["Tareas"] if t["status"] != "Listo"])}<br>'
            tasks_without_date = [t["name"] for t in report["Tareas"] if t["due_date"] == "No definido"]
            if tasks_without_date:
                body += f'Tareas sin fecha planificada: {", ".join(tasks_without_date)}<br>'
            body += '<br>'

    html_content = template.replace('main_title', f'Reporte de {user_name}')
    html_content = html_content.replace('header_img_link', '')
    html_content = html_content.replace('html_title', f'¡Hola, <b>{user_name}</b>!')
    html_content = html_content.replace('html_body', body)
    html_content = html_content.replace('graph_link', 'cid:image0')
    html_content = html_content.replace('call_action', '')
    html_content = html_content.replace('action_link', '')
    html_content = html_content.replace('end_text', '')
    html_content = html_content.replace('support_text', '')
    html_content = html_content.replace('support_email', '')

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    filename = os.path.join(output_dir, f"{nickname.replace(' ', '_').lower()}.html")
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(html_content)
    logger.info(f"Se ha generado correctamente el reporte HTML para {user_name} en {filename}")

    return filename

## Envío del reporte mediante correo electrónico 

#### Función para enviar el correo

In [ ]:
# Función para enviar el correo usando Graph API
def send_email(to_email, subject, html_content, image_paths):

    # Preparar las imágenes en base64
    attached_images = []
    for i, image_path in enumerate(image_paths):
        with open(image_path, 'rb') as img_file:
            encoded_image = base64.b64encode(img_file.read()).decode('utf-8')
            attached_images.append({
                "@odata.type": "#microsoft.graph.fileAttachment",
                "name": os.path.basename(image_path),
                "contentType": "image/png",
                "contentId": 'image0',
                "contentBytes": encoded_image,
            })

    # Preparar el cuerpo del correo
    email_msg = {
        "message": {
            "subject": subject,
            "body": {
                "contentType": "HTML",
                "content": html_content
            },
            "toRecipients": [
                {
                    "emailAddress": {
                        "address": to_email
                    }
                }
            ],
            "attachments": attached_images
        },
        "saveToSentItems": "true"
    }

    headers = {
        'Authorization': 'Bearer ' + result['access_token']
    }

    endpoint = GRAPH_API_ENDPOINT 

    try:
        response = requests.post(endpoint, headers=headers, json=email_msg)
        response.raise_for_status()  # Raise an exception if request fails

        if response.status_code == 202:
            logger.info(f"Email enviado a {user_email}.")
        else:
            logger.exception(f"Email no enviado.")

    except requests.exceptions.RequestException as e:
        logger.exception("Ocurrió un error al enviar el email.")

#### Envio de correo HTML por usuario

In [ ]:
# Generar un HTML por cada usuario
template_path = 'assets/template.html'
output_dir = './reports'

for user_name, reports in user_reports.items():
    if user_name:
        nickname = reports[0]['Usuario']['nickname']
        user_email = reports[0]['Usuario']['email']
        if not user_email:
            logger.info(f"No se ha encontrado el correo electrónico de {user_name}, se omite el envío del correo.")
            continue
        # Generar gráfico
        graph_path = generate_graph(reports, user_name, nickname, output_dir)
        # Ruta del HTML
        html_path = generate_html(user_name, nickname, reports, graph_path, template_path, output_dir)
        # Leer el contenido del HTML
        with open(html_path, 'r', encoding='utf-8') as file:
            html_content = file.read()
        send_email(user_email, f"Reporte de {user_name}", html_content, [graph_path])